In [6]:
import numpy as np
from itertools import combinations
import time

In [7]:
def is_feasible(solution, A, b, eps):
    b_est = A@solution
    sse = sum([(x-y)**2 for x,y in zip(b, b_est)])
    return sse < eps

In [8]:
def brute_force_mrvls(A, b, eps):
    n = A.shape[1] #broj kolona
    best_subset = None
    min_nonzero_vars = n + 1 
    _, res,_,_ = np.linalg.lstsq(A, b, rcond=None)

    for r in range(1, n + 1):
        # za svaki moguci broj izabranih kolona biramo podmatrice izborom kolona
        for cols in combinations(range(n), r):
            A_sub = A[:, cols]  # uzimamo podmatricu za izabrane kolone
            try:
                # proveravamo da li je sistem resiv
                x_sub = np.linalg.lstsq(A_sub, b, rcond=None)[0]
                # proveravamo da li je resenje dovoljno dobro
                if is_feasible(x_sub, A_sub, b, res*1.01):
                    if len(cols) < min_nonzero_vars:
                        best_subset = cols
                        min_nonzero_vars = len(cols)
                        break
            except np.linalg.LinAlgError:
                # ako je sistem neresiv, preskociti podmatricu
                continue

    if best_subset is not None:
        x_optimal = np.zeros(n)
        x_sub = np.linalg.lstsq(A[:, best_subset], b, rcond=None)[0]
        x_optimal[list(best_subset)] = x_sub
        return (x_optimal, min_nonzero_vars)
    else:
        return None


In [12]:
times = []
results = []
for i in range(5):
    A = np.load(f'../data/small_data/{i}_A.npy')
    b = np.load(f'../data/small_data/{i}_b.npy')
    start = time.time()
    _, res,_,_ = np.linalg.lstsq(A, b, rcond=None)
    (x,n) = brute_force_mrvls(A, b, res*1.01)
    end = time.time()
    times.append(end-start) #sec
    results.append(n)

In [13]:
times

[0.0014383792877197266,
 0.009335517883300781,
 0.03735613822937012,
 0.16786432266235352,
 1.7865383625030518]

In [14]:
results

[5, 5, 7, 8, 13]

In [15]:
np.save('../tests/bf_times', np.array(times))